In [1]:
using Pkg
Pkg.activate(".")

using Revise, PtyLab, TestImages, ImageShow, IndexFunArrays, FFTW, HDF5, Noise, FourierTools, CUDA
using EllipsisNotation
FFTW.set_num_threads(12)

  Activating environment at `~/PtyLab.jl/examples/Project.toml`
┌ Info: Precompiling PtyLab [81acdcc5-82ff-4242-81ec-b83d268fd5e7]
└ @ Base loading.jl:1342


## Object

In [2]:
img_abs = Float32.(testimage("fabio_gray_512"))[200:400, 200:400]
img_phase = Float32.(testimage("resolution_test_512"))[200:400, 200:400]
object = img_abs .* cispi.(2 .* img_phase)

complex_show(object);

## Random Grid

In [3]:
grid_size = size(object)
tile_size = (100, 100)

grr = PtyLab.grid_regular_rand(grid_size, tile_size, (15, 15), 30);
@show grr.overlap
show_grid(grr, only_points=false);

grr.overlap = (0.9299999999999999, 0.9299999999999999)


## Probe

In [4]:
probe = IndexFunArrays.gaussian(Float32, tile_size, scale=0.010) .* cis.(Float32(2π) .* 
     4 .* gaussian(Float32, tile_size, scale=0.003));

complex_show(probe);

## Simulate and store as dataset

In [5]:
ptychogram = zeros(Float32, (tile_size..., length(grr.tiles)));
p = Params()
o2d, d2o = Fraunhofer(probe, fftshiftFlag=true);

for (i, t) in enumerate(grr.tiles)
    ptychogram[:, :, i] = poisson(abs2.(o2d(view(object, t.i₁:t.i₂,  t.j₁:t.j₂) .* probe)), 1000)
end


lambda = 633f-9
z = 50f-3
dxd = 10f-6
Nd = size(ptychogram, 1)
dxo = lambda * z / (Nd * dxd)

fid_new = h5open("simulated_ptychography.hdf5", "w");
fid_new["Nd"] = Nd
fid_new["No"] = size(img_abs, 1)
fid_new["dxd"] = 10f-6
fid_new["encoder"] = PtyLab.encoder(grr, dxo, offset=(50, 50))
fid_new["wavelength"] = lambda
fid_new["entrancePupilDiameter"] = dxo * 30
fid_new["zo"] = z
fid_new["ptychogram"] = ptychogram
close(fid_new)
#@view_image ptychogram;

## Load dataset again

In [86]:
experimentalData = ExperimentalDataCPM("simulated_ptychography.hdf5");

reconstruction = ReconstructionCPM(experimentalData, cuda=false);
reconstruction = PtyLab.initializeObjectProbe!(reconstruction);

params2 = Params()

┌ Info: Reading HDF5.File: (read-only) simulated_ptychography.hdf5 was successful
└ @ PtyLab /home/fxw/PtyLab.jl/src/ExperimentalData.jl:55
┌ Warning: Currently No = 4 * Np, just as guess, should be improved
└ @ PtyLab /home/fxw/PtyLab.jl/src/utils_calc.jl:91


Params
  fftshiftFlag: Bool false
  transposePtychogram: Bool true
  intensityConstraint: PtyLab.IntensityConstraintStandard <: PtyLab.IntensityConstraint
  propagatorType: Fraunhofer (function of type typeof(Fraunhofer))
  randPositionOrder: Bool true
  comStabilizationSwitch: Bool true


In [7]:
engine = PtyLab.zPIE()

zPIE{Float32}
  betaProbe: Float32 0.25f0
  betaObject: Float32 0.25f0
  numIterations: Int64 50
  DoF: Float32 1.0f0
  zPIEgradientStepSize: Float32 100.0f0
  zPIEfriction: Float32 0.7f0
  focusObject: Bool true
  zMomentum: Float32 0.0f0


### On CPU

In [413]:
engine.zPIEgradientStepSize = 100

100

In [411]:
@show reconstruction.zo = 0.05 + reconstruction.DoF / 2

reconstruction = ReconstructionCPM(experimentalData, cuda=false);

#engines.
params2.fftshiftFlag = false
engine.betaProbe = 0.75f0
engine.betaObject = 0.75f0

params2.transposePtychogram = false
params2.comStabilizationSwitch = true
reconstruction = PtyLab.initializeObjectProbe!(reconstruction);
engine.numIterations = 50

CUDA.@time p, o = PtyLab.reconstruct(engine, params2, reconstruction);

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter /home/fxw/.julia/packages/ProgressMeter/Vf8un/src/ProgressMeter.jl:620
Progress: 100%|█████████████████████████████████████████| Time: 0:00:09
  iter:  50
  zo:    -25.145662


  9.539818 seconds (4.92 M CPU allocations: 808.826 MiB, 0.60% gc time)


In [414]:
complex_show(o[:, :, 1,1,1,1]);

In [406]:
complex_show(p[:, :, 1,1,1,1]);